In [6]:
# Check activated conda env
!conda env list
# !conda info

# conda environments:
#
base                     /home/fglauda/miniconda3
group-assur              /home/fglauda/miniconda3/envs/group-assur
jane                  *  /home/fglauda/miniconda3/envs/jane
sia                      /home/fglauda/miniconda3/envs/sia
smart-d                  /home/fglauda/miniconda3/envs/smart-d



In [85]:
from bs4 import BeautifulSoup
# import datetime
from datetime import date, timedelta, datetime
import random
import requests

In [88]:
dt_start = date(2021, 1, 1)
dt_end = date(2022, 1, 1)

dt_start = datetime.strptime('2021-01-01', '%Y-%m-%d')
dt_end = datetime.strptime('2022-01-01', '%Y-%m-%d')


def get_random_date(dt_start: datetime, dt_end: datetime) -> datetime:
    """Generate random date between 2 dates

    Args:
        dt_start: start date (included)
        dt_end: end date (excluded)

    Returns: A random date between the 2 boundaries

    """

    time_between_dates = dt_end - dt_start
    delta_end_start = time_between_dates.days
    random_number_of_days = random.randrange(delta_end_start)

    return dt_start + timedelta(days=random_number_of_days)

In [89]:
dt_random = get_random_date(dt_start, dt_end)

In [91]:
print(dt_random)
print(dt_random.strftime("%Y"))
print(dt_random.strftime("%m"))
print(dt_random.strftime("%d"))

2021-10-21 00:00:00
2021
10
21


In [22]:
d_base_url = {"Towards Data Science": "https://towardsdatascience.com/archive"}

In [30]:
year, month, day = dt_random.strftime('%Y'), dt_random.strftime('%m'), dt_random.strftime('%d')
url = f"{d_base_url.get('Towards Data Science')}/{year}/{month}/{day}"

In [31]:
response = requests.get(url, allow_redirects=True)

In [34]:
page = response.content

In [35]:
# cardChromeless
soup = BeautifulSoup(page, 'html.parser')

In [36]:
articles = soup.find_all(
    "div",
    class_="streamItem streamItem--postPreview js-streamItem"
)

In [47]:
len(articles)

32

In [52]:
l_links = []

for article in articles:
    link = article.find_all("a")[4]['href']
    l_links.append(link)


In [53]:
l_links

['https://towardsdatascience.com/the-complete-guide-to-docker-volumes-1a06051d2cce?source=collection_archive---------0-----------------------',
 'https://towardsdatascience.com/wide-to-long-data-how-and-when-to-use-pandas-melt-stack-and-wide-to-long-7c1e0f462a98?source=collection_archive---------1-----------------------',
 'https://towardsdatascience.com/uber-data-scientist-interview-experience-78305114540c?source=collection_archive---------2-----------------------',
 'https://towardsdatascience.com/how-to-remove-non-stationarity-in-time-series-forecasting-563c05c4bfc7?source=collection_archive---------3-----------------------',
 'https://towardsdatascience.com/modernize-your-sinful-python-code-with-beautiful-type-hints-4e72e98f6bf1?source=collection_archive---------4-----------------------',
 'https://towardsdatascience.com/5-different-ways-to-deploy-your-machine-learning-model-with-aws-bd676ab5f8d4?source=collection_archive---------5-----------------------',
 'https://towardsdatascie

In [54]:
# for link in l_links:

link = l_links[0]
response = requests.get(link, allow_redirects=True)

In [65]:
soup = BeautifulSoup(response.content, 'html.parser')
article = soup.find("section")
title = article.find("h1").contents[0]

In [76]:
l_links

['https://towardsdatascience.com/the-complete-guide-to-docker-volumes-1a06051d2cce?source=collection_archive---------0-----------------------',
 'https://towardsdatascience.com/wide-to-long-data-how-and-when-to-use-pandas-melt-stack-and-wide-to-long-7c1e0f462a98?source=collection_archive---------1-----------------------',
 'https://towardsdatascience.com/uber-data-scientist-interview-experience-78305114540c?source=collection_archive---------2-----------------------',
 'https://towardsdatascience.com/how-to-remove-non-stationarity-in-time-series-forecasting-563c05c4bfc7?source=collection_archive---------3-----------------------',
 'https://towardsdatascience.com/modernize-your-sinful-python-code-with-beautiful-type-hints-4e72e98f6bf1?source=collection_archive---------4-----------------------',
 'https://towardsdatascience.com/5-different-ways-to-deploy-your-machine-learning-model-with-aws-bd676ab5f8d4?source=collection_archive---------5-----------------------',
 'https://towardsdatascie

In [79]:
import unicodedata
from typing import Dict, List

def get_article(l_links: List[str]) -> pd.DataFrame:
    """ Extract medium article content from a list of urls

    Args:
        l_links: list of medium url

    Returns:
        Dataframe containing the extracted text
    """

    l_articles = []

    for link in l_links:
        try:
            d_article = {}

            # Parse web page
            data = requests.get(link)
            soup = BeautifulSoup(data.content, 'html.parser')

            # Get title
            title = soup.findAll('title')[0]
            title = title.get_text()
            d_article['title'] = unicodedata.normalize('NFKD', title)
            print(f"Scraping {title}")

            # Get author
            author = soup.findAll('meta', {"name": "author"})[0]
            author = author.get('content')
            d_article['author'] = unicodedata.normalize('NFKD', author)

            # Set link
            d_article['link'] = link

            # Get page text
            paras = soup.findAll('p')
            text = ''
            nxt_line = '\n'
            for para in paras:
                text += unicodedata.normalize('NFKD',para.get_text()) + nxt_line
            d_article['text'] = text

            # Update list
            l_articles.append(d_article)

        except:
            # for exceptions caused due to change of format on that page
            continue

    return pd.DataFrame(l_articles)

In [80]:
l_articles = get_article(l_links)

The Complete Guide to Docker Volumes | by Mahbub Zaman | Towards Data Science
Pandas Melt(), Stack() and wide_to_long() For Converting Columns into Rows | Towards Data Science
Uber Data Scientist Interview Experience | by Aqeel Anwar | Towards Data Science
How to Remove Non-Stationarity in Time Series Forecasting | by Bex T. | Towards Data Science
Modernize Your Sinful Python Code with Beautiful Type Hints | by Eirik Berge | Towards Data Science
5 Different Ways to Deploy your Machine Learning Model with AWS | by Kyle Gallatin | Towards Data Science
10 Python Modules for Full Stack Automation and Testing | by Behic Guven | Towards Data Science
Side-by-side comparison of strings in Python | by Leo van der Meulen | Towards Data Science
Decision Trees and Random Forests — Explained with Python Implementation. | by Puja P. Pathak | Towards Data Science
Deepmind AlphaFold2: Highly accurate protein structure prediction | by Mostafa Ibrahim | Towards Data Science
A Practical Introduction to S

In [83]:
import pandas as pd

pd.DataFrame(l_articles)

,title,author,link,text
0,The Complete Guide to Docker Volumes | by Mahb...,Mahbub Zaman,https://towardsdatascience.com/the-complete-gu...,"Towards Data Science\nJul 17, 2021\nSave\nThe ..."
1,"Pandas Melt(), Stack() and wide_to_long() For ...",Susan Maina,https://towardsdatascience.com/wide-to-long-da...,"Towards Data Science\nJul 17, 2021\nSave\nResh..."
2,Uber Data Scientist Interview Experience | by ...,Aqeel Anwar,https://towardsdatascience.com/uber-data-scien...,"Towards Data Science\nJul 17, 2021\nSave\nIn t..."
3,How to Remove Non-Stationarity in Time Series ...,Bex T.,https://towardsdatascience.com/how-to-remove-n...,"Towards Data Science\nJul 17, 2021\nSave\nUnli..."
4,Modernize Your Sinful Python Code with Beautif...,Eirik Berge,https://towardsdatascience.com/modernize-your-...,"Towards Data Science\nJul 17, 2021\nSave\nIn t..."
5,5 Different Ways to Deploy your Machine Learni...,Kyle Gallatin,https://towardsdatascience.com/5-different-way...,"Towards Data Science\nJul 17, 2021\nSave\nAs a..."
6,10 Python Modules for Full Stack Automation an...,Behic Guven,https://towardsdatascience.com/10-python-modul...,"Towards Data Science\nJul 17, 2021\nSave\nIn t..."
7,Side-by-side comparison of strings in Python |...,Leo van der Meulen,https://towardsdatascience.com/side-by-side-co...,"Towards Data Science\nJul 17, 2021\nSave\nCurr..."
8,Decision Trees and Random Forests — Explained ...,Puja P. Pathak,https://towardsdatascience.com/decision-trees-...,"Towards Data Science\nJul 17, 2021\nSave\nA De..."
9,Deepmind AlphaFold2: Highly accurate protein s...,Mostafa Ibrahim,https://towardsdatascience.com/deepmind-alphaf...,"Towards Data Science\nJul 17, 2021\nSave\n2 da..."


In [92]:
df = pd.read_parquet("../data/01_raw/medium_extract.parquet")

In [93]:
df

,title,author,link,text,day
0,3 Tools to Track and Visualize the Execution o...,Khuyen Tran,https://towardsdatascience.com/3-tools-to-trac...,"Towards Data Science\nApr 29, 2021\nSave\nHave...",2021-04-29
1,3 Beginner Mistakes I’ve Made in My Data Scien...,Arunn Thevapalan,https://towardsdatascience.com/3-beginner-mist...,"Towards Data Science\nApr 29, 2021\nSave\n“You...",2021-04-29
2,Data Apps with Python’s Streamlit | by Thiago ...,Thiago Carvalho,https://towardsdatascience.com/data-apps-with-...,"Towards Data Science\nApr 29, 2021\nSave\nStre...",2021-04-29
3,How to Use SQL LEAD and LAG Functions | by Mad...,Madison Schott,https://towardsdatascience.com/how-to-use-sql-...,"Towards Data Science\nApr 29, 2021\nSave\nIf y...",2021-04-29
4,Autoencoder For Denoising Images | by Michel K...,"Michel Kana, Ph.D",https://towardsdatascience.com/autoencoder-for...,"Towards Data Science\nApr 29, 2021\nSave\nIn t...",2021-04-29
5,Predict Next Month Transactions with RFM Featu...,Aigerim Shopenova,https://towardsdatascience.com/predict-next-mo...,"Towards Data Science\nApr 29, 2021\nSave\n“No ...",2021-04-29
6,Don’t Be Fooled by the Hype Python’s Got | by ...,Iyar Lin,https://towardsdatascience.com/dont-be-fooled-...,"Towards Data Science\nApr 29, 2021\nSave\nIf y...",2021-04-29
7,"Creating, editing, and merging ONNX pipelines....",Maurits Kaptein,https://towardsdatascience.com/creating-editin...,"Towards Data Science\nApr 29, 2021\nSave\nOver...",2021-04-29
8,Searching places and their geographic details ...,Sourabh Jain,https://towardsdatascience.com/searching-place...,"Towards Data Science\nApr 29, 2021\nSave\nThe ...",2021-04-29
9,Optimization of a weekly production plan with ...,Soulard Baptiste,https://towardsdatascience.com/optimization-of...,"Towards Data Science\nApr 29, 2021\nSave\nWe s...",2021-04-29
